<a href="https://colab.research.google.com/github/Kevin091295/Finetune_Resume_Q-A/blob/main/Complete_Resume_FineTune_kaggleData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import os

zip_file = "train_json.json.zip"  # Replace with the actual ZIP file name

# Extract the ZIP file
with zipfile.ZipFile(zip_file, "r") as zip_ref:
    zip_ref.extractall("extracted")  # Extract to a folder named 'extracted'

# List extracted files
os.listdir("extracted")


['train_json.json']

In [ ]:
!pip install Pandas
import pandas as pd
import json

json_file = "extracted/train_json.json"  # Replace with the actual extracted JSON filename

# Load JSON file
with open(json_file, "r") as file:
    data = json.load(file)

# Convert to Pandas DataFrame
df = pd.DataFrame(data)

print(df.head())  # Display first 5 records



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
                                             context  \
0  JT_Walkin Data Entry Operator (night Shift),CO...   
1  JT_Work Based Onhome Based Part Time,CO_find l...   
2  JT_Pl,sql Developer - SQL,CO_Softtech Career I...   
3  JT_Manager,ad,partner - Indirect Tax - CA,CO_O...   
4  JT_JAVA Technical Lead (6-8 yrs) -,CO_Spire Te...   

                                                 qas  
0  [{'id': '1', 'is_impossible': False, 'question...  
1  [{'id': '6', 'is_impossible': False, 'question...  
2  [{'id': '11', 'is_impossible': False, 'questio...  
3  [{'id': '16', 'is_impossible': False, 'questio...  
4  [{'id': '21', 'is_impossible': False, 'questio...  


In [ ]:
!pip install transformers datasets peft bitsandbytes trl accelerate pdfplumber torch
import os
import torch
import pandas as pd
import json
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
)
from trl import SFTTrainer


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
# Process the data from the already loaded DataFrame
def prepare_training_data(data):
    """Process the data into the format needed for training."""
    train_data = []

    # Check if data is a list of dictionaries with 'context' and 'qas' keys
    if isinstance(data, list):
        print("Processing list of examples...")

        for item in data:
            resume_text = item.get('context', '')
            qas = item.get('qas', [])

            for qa in qas:
                question = qa.get('question', '')
                # Handle answers which might be in a list
                answers = qa.get('answers', [])
                if answers and isinstance(answers, list):
                    answer = answers[0].get('text', '') if answers else ''
                else:
                    answer = ''

                # Skip examples with missing data
                if not resume_text or not question or not answer:
                    continue

                # Format according to Llama 3 instruction format
                formatted_prompt = f"""<|begin_of_text|><|user|>
I have a resume:
{resume_text}

Question: {question}<|end_of_text|>
<|assistant|>
{answer}<|end_of_text|>"""

                train_data.append({
                    "text": formatted_prompt,  # Full formatted text for training
                })

    # Print some information about the processed data
    print(f"Prepared {len(train_data)} examples for training")

    # Show a sample prompt if available
    if train_data:
        print("\nSample prompt:")
        sample_prompt = train_data[0]["text"]
        # Truncate if too long
        if len(sample_prompt) > 500:
            print(f"{sample_prompt[:500]}...")
        else:
            print(sample_prompt)

    return train_data

# Prepare the dataset
print("Preparing training data...")

train_data = prepare_training_data(data)
train_data = train_data[:3000]

Preparing training data...
Processing list of examples...
Prepared 90000 examples for training

Sample prompt:
<|begin_of_text|><|user|>
I have a resume:
JT_Walkin Data Entry Operator (night Shift),CO_MM Media Pvt Ltd,JR_Fresher Keyskills English Typing Part Time Data Entry Selection Process Desired Candidate Profile ,JL_Bangalore,ED_Postgraduate,EX_0 - 1 yrs,SK_ITES,IN_Media , Entertainment , Internet,PR_1,50,000 - 2,25,000 PA

Question: What is job provided by MM Media Pvt Ltd company?<|end_of_text|>
<|assistant|>
Walkin Data Entry Operator (night Shift)<|end_of_text|>


In [ ]:
# Convert to Dataset format
train_df = pd.DataFrame(train_data)
dataset = Dataset.from_pandas(train_df)

# Split into train/validation sets
train_val_split = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_val_split['train']
val_dataset = train_val_split['test']

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

Training set size: 2700
Validation set size: 300


In [ ]:
# Configure QLoRA for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
!pip install huggingface_hub


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
from huggingface_hub import login

login("HF_Token")  # Use your actual token


In [ ]:
# Set parameters
MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
OUTPUT_DIR = "./resume-agent-model"
# Initialize tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading tokenizer...


In [ ]:
# Load model with quantization config

!pip install --upgrade typing_extensions
!pip install scipy
!pip install --upgrade bitsandbytes



print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
)


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
  Using cached bitsandbytes-0.45.3-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
Using cached bitsandbytes-0.45.3-py3-none-manylinux_2_24_x86_64.whl (76.1 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.41.1
    Uninstalling bitsandbytes-0.41.1:
      Successfully uninstalled bitsandbytes-0.41.1

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Prepare model for LoRA training
model = prepare_model_for_kbit_training(model)

In [ ]:
# Configure LoRA
print("Configuring LoRA...")
lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,  # Alpha parameter for LoRA
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)

# Display trainable parameters
model.print_trainable_parameters()

Configuring LoRA...
trainable params: 24,313,856 || all params: 3,237,063,680 || trainable%: 0.7511


In [ ]:
# Define training arguments - tuned for T4 GPU with limited memory
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=1,  # Small batch size for T4
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,  # Increase this to simulate larger batch sizes
    gradient_checkpointing=True,  # Enable gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    evaluation_strategy="steps",
    eval_steps=50,  # Check validation loss every 50 steps
    save_strategy="steps",
    save_steps=50,  # Save model every 50 steps
    load_best_model_at_end=True,
    report_to="none",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Configure trainer
print("Configuring trainer...")
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=training_args,
    tokenizer=tokenizer,

     # Lower sequence length for T4 memory constraints
     # Enable packing for more efficient training
)

Configuring trainer...


/tmp/ipykernel_814/2115083799.py:3: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/2700 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2700 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2700 [00:00<?, ? examples/s]

...

Converting eval dataset to ChatML:   0%|          | 0/300 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Start training
print("Starting training...")
trainer.train()

Starting training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
50,0.946800,1.426705
100,0.752200,1.176046
150,0.650700,1.060073
200,0.596300,0.973425
250,0.480400,0.842787
300,0.451700,0.750723
350,0.892200,0.630819
400,0.871900,0.564628
450,0.753000,0.500843
500,0.606400,0.422408


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=1011, training_loss=0.5464386601741661, metrics={'train_runtime': 2297.366, 'train_samples_per_second': 3.526, 'train_steps_per_second': 0.44, 'total_flos': 2.9577895729895424e+16, 'train_loss': 0.5464386601741661})

In [ ]:
print("Saving model...")
trainer.save_model(OUTPUT_DIR)
print(f"Model saved to {OUTPUT_DIR}")

Saving model...
Model saved to ./resume-agent-model


In [ ]:
!pip install huggingface_hub transformers

from huggingface_hub import HfApi, HfFolder

model.push_to_hub("KevinPanchal/FineTune_Resume_Kaggle")
tokenizer.push_to_hub("KevinPanchal/FineTune_Resume_Kaggle")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/KevinPanchal/FineTune_Resume_Kaggle/commit/3c2516068661d35678d37da7cdc0b09d0fcb23a8', commit_message='Upload tokenizer', commit_description='', oid='3c2516068661d35678d37da7cdc0b09d0fcb23a8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/KevinPanchal/FineTune_Resume_Kaggle', endpoint='https://huggingface.co', repo_type='model', repo_id='KevinPanchal/FineTune_Resume_Kaggle'), pr_revision=None, pr_num=None)